## Comparison of Model 2 created with basic RFM model

In [19]:
# Import The Libraries
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, scipy as sp, warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.metrics import silhouette_samples,silhouette_score

In [4]:
filename = 'Method2_Model.sav'
kmeans = pickle.load(open(filename, 'rb'))

In [6]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price','RFM_Score'],axis = 'columns',inplace=True)
final_data.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data.columns.values.tolist()
scaler = MinMaxScaler() 
final_data[numeric_columns] = scaler.fit_transform(final_data[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

# for i in range_values:    
#     kmeans = KMeans(n_clusters= i)
#     kmeans.fit(final_data)
#     scores_2.append(kmeans.inertia_)
# kmeans = KMeans(3)
# kmeans.fit(final_data)
labels = kmeans.labels_
y_kmeans = kmeans.fit_predict(final_data)
df_cluster_dr = pd.concat([final_data_new, pd.DataFrame({'cluster_final':labels})], axis = 1)
print(df_cluster_dr['cluster_final'].value_counts())

0    3594
2    1503
1     781
Name: cluster_final, dtype: int64


In [3]:
dataset=pd.read_excel('C:/Personal/NUIG/Project-DA/online_retail_II.xlsx',sheet_name=None)
data_before = dataset['Year 2009-2010']
data_after = dataset['Year 2010-2011']

In [73]:
#select one customer in both datasets
#get the cluster of that customer
#recommend top 20 customer wala thing
#remove all descriptions he already bought
#check in after dataset how many matches

In [28]:
#remove all customers with NA customer ID
data_before = data_before[data_before['Customer ID'].notna()]
data_after = data_after[data_after['Customer ID'].notna()]
cust_before = (data_before.sample(n=20)['Customer ID']).tolist()

In [29]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr[df_cluster_dr['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 1:
            d = kmeans.transform(final_data)[:, 0]
        elif cluster['cluster_final'].item() == 0:
            d = kmeans.transform(final_data)[:, 1]
        else:
            d = kmeans.transform(final_data)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr[df_cluster_dr.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr[df_cluster_dr['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 32.35 0 [0]
Description matching rate 23.64 0 [0]
Description matching rate 26.51 0 [0]
Description matching rate 33.33 0 [0]
Description matching rate 10.17 0 [0]
Customer Not present
Customer Not present
Description matching rate 9.65 0 [0]
Description matching rate 33.72 0 [0]
Description matching rate 19.19 0 [0]
Description matching rate 100.0 2 [2]
Description matching rate 42.86 0 [0]
Customer Not present
Description matching rate 12.15 0 [0]
Customer Not present
Customer Not present
Customer Not present
Customer Not present


In [30]:
sum(percent_match)/len(percent_match)

0.31233345596990447

In [20]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data_1 = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
final_data_1.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price',0,1,2,3,4,5],axis = 'columns',inplace=True)
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data_1.columns.values.tolist()
scaler = MinMaxScaler() 
final_data_1[numeric_columns] = scaler.fit_transform(final_data_1[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

for i in range_values:
    
    kmeans_1 = KMeans(n_clusters= i)
    kmeans_1.fit(final_data_1)
    scores_2.append(kmeans_1.inertia_)
kmeans_1 = KMeans(3)
kmeans_1.fit(final_data_1)
labels_1 = kmeans_1.labels_
y_kmeans_1 = kmeans_1.fit_predict(final_data_1)
df_cluster_dr_1 = pd.concat([final_data_1_new, pd.DataFrame({'cluster_final':labels_1})], axis = 1)
print(df_cluster_dr_1['cluster_final'].value_counts())
silhouette_avg = silhouette_score(final_data, y_kmeans)
print('score de silhouette: {:<.3f}'.format(silhouette_avg))

2    2129
0    1994
1    1755
Name: cluster_final, dtype: int64
score de silhouette: 0.680


In [31]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 0:
            d = kmeans_1.transform(final_data_1)[:, 0]
        elif cluster['cluster_final'].item() == 1:
            d = kmeans_1.transform(final_data_1)[:, 1]
        else:
            d = kmeans_1.transform(final_data_1)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr_1[df_cluster_dr_1.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 4.41 0 [2]
Description matching rate 9.09 0 [2]
Description matching rate 9.64 0 [2]
Description matching rate 11.11 0 [2]
Description matching rate 2.54 0 [2]
Customer Not present
Customer Not present
Description matching rate 2.63 0 [2]
Description matching rate 18.6 0 [2]
Description matching rate 5.05 0 [2]
Description matching rate 100.0 1 [0]
Description matching rate 7.14 0 [2]
Customer Not present
Description matching rate 1.87 0 [2]
Customer Not present
Customer Not present
Customer Not present
Customer Not present


In [32]:
sum(percent_match)/len(percent_match)

0.15644860289831994

In [24]:
#read prod category and rfm data
prod_cat = pd.read_pickle('PCA_RED_DESC.pkl')
rfm = pd.read_pickle('rfm.pkl')
final_data_1 = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1_new = pd.merge(prod_cat,rfm,on='Customer ID')
final_data_1.rename(columns={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5'})
final_data_1.drop(['RFM_Segment','Customer ID','R','F','M','Final_Price',0,1,2,3,4,5,'RFM_Score'],axis = 'columns',inplace=True)
#Rescaling
from sklearn.preprocessing import MinMaxScaler
numeric_columns = final_data_1.columns.values.tolist()
scaler = MinMaxScaler() 
final_data_1[numeric_columns] = scaler.fit_transform(final_data_1[numeric_columns])
from sklearn.cluster import KMeans
scores_2 = []

range_values = range(1, 20)

for i in range_values:
    
    kmeans_1 = KMeans(n_clusters= i)
    kmeans_1.fit(final_data_1)
    scores_2.append(kmeans_1.inertia_)
kmeans_1 = KMeans(3)
kmeans_1.fit(final_data_1)
labels_1 = kmeans_1.labels_
y_kmeans_1 = kmeans_1.fit_predict(final_data_1)
df_cluster_dr_1 = pd.concat([final_data_1_new, pd.DataFrame({'cluster_final':labels_1})], axis = 1)
print(df_cluster_dr_1['cluster_final'].value_counts())

0    3594
1    1503
2     781
Name: cluster_final, dtype: int64


In [25]:
from sklearn.metrics import silhouette_samples,silhouette_score
silhouette_avg = silhouette_score(final_data, y_kmeans)
print('score de silhouette: {:<.3f}'.format(silhouette_avg))

score de silhouette: 0.680


In [33]:
percent_match = []
for cust in cust_before:
    if cust in data_after['Customer ID']:
        cluster = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        if cluster['cluster_final'].item() == 2:
            d = kmeans_1.transform(final_data_1)[:, 0]
        elif cluster['cluster_final'].item() == 0:
            d = kmeans_1.transform(final_data_1)[:, 1]
        else:
            d = kmeans_1.transform(final_data_1)[:, 2]
        #select 20 nearest points to that cluster and get their indices
        ind = np.argsort(d)[::][:30]
        #get the data of these 20 customers
        data_recom_id = df_cluster_dr_1[df_cluster_dr_1.index.isin(ind)]
        #get the desc of that customer before recommending
        data_bf = data_before[data_before['Customer ID'] == cust]['Description'].apply(str).unique()
        #results of recommendation
        data_recom = data_recom_id.merge(data_before, on='Customer ID', how='left')
        descriptions=data_recom['Description'].apply(str).unique()
        des_final = np.setdiff1d(descriptions,data_bf)
        #evaluate with data_after
        data_input = df_cluster_dr_1[df_cluster_dr_1['Customer ID'] == cust]
        data_after_1 = data_input.merge(data_after, on='Customer ID', how='left')
        descriptions_after=data_after_1['Description'].apply(str).unique()
        percent = len([x for x in descriptions_after if x in des_final])/len(descriptions_after)
        print('Description matching rate',round(percent*100,2),cluster['cluster_final'].item(),data_recom_id['cluster_final'].unique())
        percent_match.append(percent)
    else:
        print('Customer Not present')

Customer Not present
Customer Not present
Description matching rate 26.47 0 [0]
Description matching rate 23.64 0 [0]
Description matching rate 25.3 0 [0]
Description matching rate 30.56 0 [0]
Description matching rate 10.17 0 [0]
Customer Not present
Customer Not present
Description matching rate 10.53 0 [0]
Description matching rate 33.72 0 [0]
Description matching rate 16.16 0 [0]
Description matching rate 100.0 1 [1]
Description matching rate 42.86 0 [0]
Customer Not present
Description matching rate 12.15 0 [0]
Customer Not present
Customer Not present
Customer Not present
Customer Not present


In [34]:
sum(percent_match)/len(percent_match)

0.30140794683907757